# Quick Start Guide

This notebook provides a 5-minute introduction to the Hypersonic Reentry Trajectory Optimization Framework.

## 🚀 Learning Objectives

By the end of this notebook, you will:
- Understand the basic framework components
- Run your first trajectory simulation
- Create a simple visualization
- Calculate performance metrics

## 📋 Prerequisites

- Framework installed: `pip install -e .`
- Python 3.8+ with NumPy, SciPy, Matplotlib

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Add framework to path (if not installed)
sys.path.insert(0, str(Path('..') / 'src'))

# Import framework components
from hypersonic_reentry.dynamics import VehicleDynamics, VehicleState
from hypersonic_reentry.atmosphere import USStandard1976
from hypersonic_reentry.utils.constants import DEG_TO_RAD, RAD_TO_DEG

print("✅ Framework imported successfully!")
print(f"Python version: {sys.version}")

## Step 1: Create Atmosphere Model

The US Standard Atmosphere 1976 provides realistic atmospheric properties including density, temperature, and pressure.

In [ ]:
# Create atmosphere model
atmosphere = USStandard1976()

# Test atmospheric properties at different altitudes
altitudes = [0, 25000, 50000, 75000, 100000]  # meters

print("Atmospheric Properties:")
print("Alt (km)  | Temp (K) | Press (Pa) | Density (kg/m³)")
print("-" * 50)

for alt in altitudes:
    props = atmosphere.get_properties(alt)
    print(f"{alt/1000:8.1f} | {props['temperature']:8.1f} | {props['pressure']:10.2e} | {props['density']:11.4e}")

## Step 2: Define Vehicle Parameters

Configure a representative hypersonic reentry vehicle with realistic parameters.

In [ ]:
# Define vehicle parameters
vehicle_params = {
    'mass': 5000.0,              # kg - vehicle mass
    'reference_area': 15.0,      # m² - reference area for aerodynamics
    'drag_coefficient': 1.2,     # dimensionless drag coefficient
    'lift_coefficient': 0.8,     # dimensionless lift coefficient
    'ballistic_coefficient': 400.0,  # kg/m² - mass/drag ratio
    'nose_radius': 0.5,          # m - nose radius for heating
    'length': 10.0,              # m - vehicle length
    'diameter': 2.0              # m - maximum diameter
}

print("Vehicle Configuration:")
for param, value in vehicle_params.items():
    print(f"{param:20}: {value:8.1f}")

# Calculate derived parameters
print("\nDerived Parameters:")
print(f"{'Ballistic Coeff':20}: {vehicle_params['ballistic_coefficient']:8.1f} kg/m²")
print(f"{'L/D Ratio':20}: {vehicle_params['lift_coefficient']/vehicle_params['drag_coefficient']:8.2f}")

## Step 3: Create Vehicle Dynamics

The VehicleDynamics class combines the vehicle parameters with the atmosphere model to solve the equations of motion.

In [ ]:
# Create vehicle dynamics system
dynamics = VehicleDynamics(vehicle_params, atmosphere_model=atmosphere)

print("✅ Vehicle dynamics system created")
print(f"Vehicle type: Hypersonic reentry vehicle")
print(f"Mass: {vehicle_params['mass']} kg")
print(f"Reference area: {vehicle_params['reference_area']} m²")

## Step 4: Define Initial Conditions

Set up initial conditions for a typical reentry from Low Earth Orbit (LEO).

In [ ]:
# Define initial conditions (Kennedy Space Center reentry)
initial_state = VehicleState(
    altitude=120000.0,                    # 120 km altitude
    latitude=28.5 * DEG_TO_RAD,          # Kennedy Space Center latitude
    longitude=-80.6 * DEG_TO_RAD,        # Kennedy Space Center longitude  
    velocity=7800.0,                     # 7.8 km/s - typical LEO velocity
    flight_path_angle=-1.5 * DEG_TO_RAD, # -1.5° shallow entry
    azimuth=90.0 * DEG_TO_RAD,          # Eastward flight
    time=0.0                             # Start time
)

print("Initial Conditions:")
print(f"Altitude:           {initial_state.altitude/1000:8.1f} km")
print(f"Latitude:           {initial_state.latitude*RAD_TO_DEG:8.1f} °")
print(f"Longitude:          {initial_state.longitude*RAD_TO_DEG:8.1f} °")
print(f"Velocity:           {initial_state.velocity:8.1f} m/s")
print(f"Flight path angle:  {initial_state.flight_path_angle*RAD_TO_DEG:8.1f} °")
print(f"Azimuth:            {initial_state.azimuth*RAD_TO_DEG:8.1f} °")

## Step 5: Run Trajectory Simulation

Integrate the equations of motion to simulate the reentry trajectory.

In [ ]:
# Run trajectory simulation
print("🚀 Starting trajectory simulation...")

# Define simulation parameters
time_span = (0.0, 2000.0)  # 2000 seconds simulation
time_step = 1.0            # 1 second time step

# Integrate trajectory
trajectory = dynamics.integrate_trajectory(
    initial_state=initial_state,
    time_span=time_span,
    time_step=time_step
)

print(f"✅ Simulation complete!")
print(f"Simulation time: {trajectory['time'][-1]:.1f} seconds")
print(f"Data points: {len(trajectory['time'])}")

# Display final conditions
print("\nFinal Conditions:")
print(f"Final altitude:     {trajectory['altitude'][-1]/1000:8.1f} km")
print(f"Final velocity:     {trajectory['velocity'][-1]:8.1f} m/s")
print(f"Downrange:          {trajectory['downrange'][-1]/1000:8.1f} km")
print(f"Flight time:        {trajectory['time'][-1]/60:8.1f} minutes")

## Step 6: Create Visualizations

Generate plots to understand the trajectory behavior.

In [ ]:
# Create comprehensive trajectory plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Hypersonic Reentry Trajectory Analysis', fontsize=16, fontweight='bold')

# 1. Altitude vs Time
axes[0,0].plot(trajectory['time']/60, trajectory['altitude']/1000, 'b-', linewidth=2)
axes[0,0].set_xlabel('Time (minutes)')
axes[0,0].set_ylabel('Altitude (km)')
axes[0,0].set_title('Altitude Profile')
axes[0,0].grid(True, alpha=0.3)

# 2. Velocity vs Time  
axes[0,1].plot(trajectory['time']/60, trajectory['velocity']/1000, 'r-', linewidth=2)
axes[0,1].set_xlabel('Time (minutes)')
axes[0,1].set_ylabel('Velocity (km/s)')
axes[0,1].set_title('Velocity Profile')
axes[0,1].grid(True, alpha=0.3)

# 3. Altitude vs Velocity (Trajectory in state space)
axes[1,0].plot(trajectory['velocity']/1000, trajectory['altitude']/1000, 'g-', linewidth=2)
axes[1,0].set_xlabel('Velocity (km/s)')
axes[1,0].set_ylabel('Altitude (km)')
axes[1,0].set_title('Trajectory Phase Plane')
axes[1,0].grid(True, alpha=0.3)

# 4. Ground Track (if longitude/latitude data available)
if 'latitude' in trajectory and 'longitude' in trajectory:
    axes[1,1].plot(trajectory['longitude']*RAD_TO_DEG, trajectory['latitude']*RAD_TO_DEG, 'purple', linewidth=2)
    axes[1,1].plot(initial_state.longitude*RAD_TO_DEG, initial_state.latitude*RAD_TO_DEG, 'go', markersize=8, label='Start')
    axes[1,1].plot(trajectory['longitude'][-1]*RAD_TO_DEG, trajectory['latitude'][-1]*RAD_TO_DEG, 'ro', markersize=8, label='End')
    axes[1,1].set_xlabel('Longitude (°)')
    axes[1,1].set_ylabel('Latitude (°)')
    axes[1,1].set_title('Ground Track')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
else:
    # Downrange vs Time as alternative
    axes[1,1].plot(trajectory['time']/60, trajectory['downrange']/1000, 'orange', linewidth=2)
    axes[1,1].set_xlabel('Time (minutes)')
    axes[1,1].set_ylabel('Downrange (km)')
    axes[1,1].set_title('Downrange Distance')
    axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Trajectory plots created successfully!")

## Step 7: Calculate Performance Metrics

Analyze the trajectory performance and key mission metrics.

In [ ]:
# Calculate performance metrics
performance_metrics = {
    'final_altitude_km': trajectory['altitude'][-1] / 1000,
    'final_velocity_ms': trajectory['velocity'][-1],
    'final_velocity_mach': trajectory['velocity'][-1] / 340,  # Approximate Mach number
    'downrange_km': trajectory['downrange'][-1] / 1000,
    'flight_time_min': trajectory['time'][-1] / 60,
    'max_altitude_km': np.max(trajectory['altitude']) / 1000,
    'min_altitude_km': np.min(trajectory['altitude']) / 1000,
    'max_velocity_ms': np.max(trajectory['velocity']),
    'min_velocity_ms': np.min(trajectory['velocity'])
}

# Add heat rate if available
if 'heat_rate' in trajectory:
    performance_metrics['max_heat_rate_MW_m2'] = np.max(trajectory['heat_rate']) / 1e6
    
# Add dynamic pressure if available  
if 'dynamic_pressure' in trajectory:
    performance_metrics['max_dynamic_pressure_kPa'] = np.max(trajectory['dynamic_pressure']) / 1000

print("\n📈 PERFORMANCE METRICS")
print("=" * 40)

print("\n🎯 Mission Performance:")
print(f"Final altitude:        {performance_metrics['final_altitude_km']:8.2f} km")
print(f"Final velocity:        {performance_metrics['final_velocity_ms']:8.1f} m/s")
print(f"Final Mach number:     {performance_metrics['final_velocity_mach']:8.2f}")
print(f"Downrange distance:    {performance_metrics['downrange_km']:8.1f} km")
print(f"Flight time:           {performance_metrics['flight_time_min']:8.1f} minutes")

print("\n📊 Trajectory Extremes:")
print(f"Max altitude:          {performance_metrics['max_altitude_km']:8.2f} km")
print(f"Min altitude:          {performance_metrics['min_altitude_km']:8.2f} km")
print(f"Max velocity:          {performance_metrics['max_velocity_ms']:8.1f} m/s")
print(f"Min velocity:          {performance_metrics['min_velocity_ms']:8.1f} m/s")

if 'max_heat_rate_MW_m2' in performance_metrics:
    print(f"Max heat rate:         {performance_metrics['max_heat_rate_MW_m2']:8.2f} MW/m²")
    
if 'max_dynamic_pressure_kPa' in performance_metrics:
    print(f"Max dynamic pressure:  {performance_metrics['max_dynamic_pressure_kPa']:8.1f} kPa")

## 🎉 Congratulations!

You've successfully:
- ✅ Created an atmospheric model
- ✅ Configured a hypersonic vehicle
- ✅ Simulated a reentry trajectory
- ✅ Visualized the results
- ✅ Calculated performance metrics

## 🚀 Next Steps

Now that you've mastered the basics, explore more advanced capabilities:

1. **Uncertainty Analysis**: `02_framework_overview.ipynb`
2. **Monte Carlo Simulation**: `03_monte_carlo_analysis.ipynb`
3. **Trajectory Optimization**: `04_trajectory_optimization.ipynb`
4. **Sensitivity Analysis**: `05_sensitivity_analysis.ipynb`

## 🔗 Additional Resources

- **User Guide**: `../USER_GUIDE.md`
- **Methodology**: `../website/methodology.md`
- **Examples**: `../examples/`
- **API Documentation**: `../docs/api/`

Happy analyzing! 🚀